In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import *

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import matplotlib.pyplot as plt
import numpy as np

from resnet import *

import pickle


/home/pratyush/miniconda3/envs/pytorch_3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import argparse
import os

# parser = argparse.ArgumentParser(description='PyTorch Cifar10 Training')
# parser.add_argument('--ngpu', type=int, default=1, help='0 = CPU.')
# parser.add_argument('--gpu_id', type=int, default=0,
#                     help='device range [0,ngpu-1]')


# args = parser.parse_args()

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
ngpu = 1
gpu_id = 1
if ngpu == 1:
    # make only devices indexed by #gpu_id visible
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.01

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torchattacks
import matplotlib.pyplot as plt

# Define transformations for the training and test sets
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Define the ResNet-18 model
# Model
print('==> Building model..')
model = ResNet18()
model = model.to(device)
if device == 'cuda':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Define the PGD attack
pgd_attack = torchattacks.PGD(model, eps=8/255, alpha=3/255, steps=7)

# Training function with adversarial training
def train(model, trainloader, optimizer, criterion, attack):
    model.train()
    train_loss = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()

        # Generate adversarial examples
        adv_inputs = attack(inputs, targets)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass with adversarial examples
        outputs = model(adv_inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Batch {batch_idx}/{len(trainloader)}: Loss = {loss.item()}')
    return train_loss / len(trainloader)

# Function to evaluate the model
def test(model, testloader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'Test Loss: {test_loss/len(testloader)}, Accuracy: {100.*correct/total}%')
    return test_loss / len(testloader), 100. * correct / total

# Training loop
num_epochs = 20
train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train_loss = train(model, trainloader, optimizer, criterion, pgd_attack)
    test_loss, test_accuracy = test(model, testloader, criterion)

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    # Adjust the learning rate
    if epoch == 10:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.01
    elif epoch == 15:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001

# Plot training and test losses
plt.figure(figsize=(10,5))
plt.title("Training and Test Losses")
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot test accuracy
plt.figure(figsize=(10,5))
plt.title("Test Accuracy")
plt.plot(test_accuracies, label="Test Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()


# Save the model parameters
torch.save(model.state_dict(), 'resnet18_cifar10_pgdTorchattack_iter7.pth')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..
Epoch 1/20
Batch 0/391: Loss = 2.3402228355407715


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torchattacks
import matplotlib.pyplot as plt

# Define transformations for the training and test sets
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Define the ResNet-18 model
# Model
print('==> Building model..')
model = ResNet18()
model = model.to(device)
if device == 'cuda':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Define the PGD attack
pgd_attack = torchattacks.PGD(model, eps=8/255, alpha=3/255, steps=20)

# Training function with adversarial training
def train(model, trainloader, optimizer, criterion, attack):
    model.train()
    train_loss = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()

        # Generate adversarial examples
        adv_inputs = attack(inputs, targets)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass with adversarial examples
        outputs = model(adv_inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Batch {batch_idx}/{len(trainloader)}: Loss = {loss.item()}')
    return train_loss / len(trainloader)

# Function to evaluate the model
def test(model, testloader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'Test Loss: {test_loss/len(testloader)}, Accuracy: {100.*correct/total}%')
    return test_loss / len(testloader), 100. * correct / total

# Training loop
num_epochs = 20
train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train_loss = train(model, trainloader, optimizer, criterion, pgd_attack)
    test_loss, test_accuracy = test(model, testloader, criterion)

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    # Adjust the learning rate
    if epoch == 10:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.01
    elif epoch == 15:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001

# Plot training and test losses
plt.figure(figsize=(10,5))
plt.title("Training and Test Losses")
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot test accuracy
plt.figure(figsize=(10,5))
plt.title("Test Accuracy")
plt.plot(test_accuracies, label="Test Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()


# Save the model parameters
torch.save(model.state_dict(), 'resnet18_cifar10_pgdTorchattack_iter20.pth')